In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
# from sklearn import datasets

In [2]:
widths = (8,7,4,13,43)
header_pd = pd.read_fwf('Labeled_data.txt', widths = widths,skiprows=7, nrows=27)
labeled_data = pd.read_csv('Labeled_data.txt', header=None, delim_whitespace=True, skiprows=37) # extract data
labeled_data.columns = header_pd.iloc[:,3]
labeled_data.head()
#shuffle data
labeled_data_sampled = labeled_data.sample(n=1000, random_state=1)

In [3]:
def weighted_mean(mag,mag_err):
    mag2 = (mag_err*mag_err) # mag err square
    mag2_inv = 1/mag2.values; # take inverse of the values
    w = pd.Series(mag2_inv) # covert it back to s series
    sw = w.sum() # sum of weights
    wmag = mag*w # multiply magnitude with weights
    wmean = wmag.sum()/sw # weighted mean
    return wmean

In [4]:
# welsh J, K statistics
def welsh_staton(mag_series,wmean):
    N = len(mag_series)
    d_i = N/(N-1)*(mag_series - wmean) # replace mean by weighted mean
    d_i1 = d_i.shift(periods=-1)
    d_i1.fillna(0, inplace = True)
    Pi = d_i*d_i1
    Pi_val = Pi.values
    Psign = np.sign(Pi_val)
    Jval = Psign*np.sqrt(np.abs(Pi_val))
    J = np.sum(Jval) 
    K1 = abs(d_i.values)/N
    K2 = np.sqrt(1/N*np.sum(d_i.values*d_i.values))
    K = np.sum(K1*K2)
    return J, K 

In [5]:
g_mean = []
g_wmean = [] # weighted mean
g_MAD = []
g_IQR = []
g_f60 = []
g_f70 = []
g_f80 = []
g_f90 = []
g_skew = []
g_kurtosis = []
g_welsh_K = []
g_welsh_J = []

r_mean = []
r_wmean = [] # weighted mean
r_MAD = []
r_IQR = []
r_f60 = []
r_f70 = []
r_f80 = []
r_f90 = []
r_skew = []
r_kurtosis = []
r_welsh_K = []
r_welsh_J = []

In [6]:
# lightcurves = {} # empty dictionary to hold dataframe for each light curve
circle_radius = 0.00028 # 1 arcsec = 0.00028 degress
t_format = "ipac_table"
table_format = "FORMAT=" + str(t_format)
flag_mask = 32768
mask = "BAD_CATFLAGS_MASK=" + str(flag_mask)
collect="COLLECTION="+"ztf_dr2"
numobs = "NOBS_MIN=20"
filter_band = "g"
label = []
SourceID =[]
start_time = time.time()
k = 0
for i in labeled_data_sampled.index:
    label.append(labeled_data_sampled.Type[i])
    SourceID.append(labeled_data_sampled.SourceID[i])
    ra = labeled_data_sampled.RAdeg[i]
    dec = labeled_data_sampled.DEdeg[i]
    circle = "POS=CIRCLE"+"+"+str(ra)+"+"+str(dec)+"+"+str(circle_radius)
    band = "BANDNAME="+ filter_band
    params = circle + "&" +  mask + "&" + numobs + "&" + collect + "&" + table_format
    
    url= "https://irsa.ipac.caltech.edu/cgi-bin/ZTF/nph_light_curves?" + params
    data = pd.read_csv(url, header=None, delim_whitespace=True, skiprows=55) # extract data
    header = pd.read_csv(url, header=None, sep='|', skiprows=50,usecols=range(1,25), nrows=1)
    data.columns = header.iloc[0].str.strip()
    lc = data
    
    dfg = lc.loc[lc["filtercode"] == "zg"]
    dfr = lc.loc[lc["filtercode"] == "zr"]
    
    if len(dfg) >0:
        N = len(dfg)
        wmean_temp = np.average(dfg.mag, weights = dfg.magerr*dfg.magerr)
#         wmean_temp = weighted_mean(dfg.mag,dfg.magerr)
        K_temp, J_temp =  welsh_staton(dfg.mag, wmean_temp )
        g_mean.append(dfg.mag.mean())
        g_wmean.append(wmean_temp) 
        deviation = abs(dfg.mag - dfg.mag.median())
        g_MAD.append(deviation.median())
        g_IQR.append(dfg.mag.quantile(0.75) - dfg.mag.quantile(0.25))
        g_f60.append(dfg.mag.quantile(0.80) - dfg.mag.quantile(0.2))
        g_f70.append(dfg.mag.quantile(0.85) - dfg.mag.quantile(0.15))
        g_f80.append(dfg.mag.quantile(0.9) - dfg.mag.quantile(0.10))
        g_f90.append(dfg.mag.quantile(0.95) - dfg.mag.quantile(0.05))
        g_skew.append(dfg.mag.skew())
        g_kurtosis.append(dfg.mag.kurtosis())
        g_welsh_J.append(J_temp)
        g_welsh_K.append(K_temp)
    else:
        g_mean.append(np.NaN)
        g_wmean.append(np.NaN) 
        g_MAD.append(np.NaN)
        g_IQR.append(np.NaN)
        g_f60.append(np.NaN)
        g_f70.append(np.NaN)
        g_f80.append(np.NaN)
        g_f90.append(np.NaN)
        g_skew.append(np.NaN)
        g_kurtosis.append(np.NaN)
        g_welsh_J.append(np.NaN)
        g_welsh_K.append(np.NaN)
        
    if len(dfr) >0:
        N = len(dfr)
#         wmean_temp = weighted_mean(dfr.mag,dfr.magerr)
        wmean_temp = np.average(dfr.mag, weights = dfr.magerr*dfr.magerr)
        K_temp, J_temp =  welsh_staton(dfr.mag, wmean_temp )
        r_mean.append(dfr.mag.mean())
        r_wmean.append(wmean_temp) 
        deviation = abs(dfr.mag - dfr.mag.median())
        r_MAD.append(deviation.median())
        r_IQR.append(dfr.mag.quantile(0.75) - dfr.mag.quantile(0.25))
        r_f60.append(dfr.mag.quantile(0.80) - dfr.mag.quantile(0.2))
        r_f70.append(dfr.mag.quantile(0.85) - dfr.mag.quantile(0.15))
        r_f80.append(dfr.mag.quantile(0.9) - dfr.mag.quantile(0.10))
        r_f90.append(dfr.mag.quantile(0.95) - dfr.mag.quantile(0.05))
        r_skew.append(dfr.mag.skew())
        r_kurtosis.append(dfr.mag.kurtosis())
        r_welsh_J.append(J_temp)
        r_welsh_K.append(K_temp)
    else:
        r_mean.append(np.NaN)
        r_wmean.append(np.NaN) 
        r_MAD.append(np.NaN)
        r_IQR.append(np.NaN)
        r_f60.append(np.NaN)
        r_f70.append(np.NaN)
        r_f80.append(np.NaN)
        r_f90.append(np.NaN)
        r_skew.append(np.NaN)
        r_kurtosis.append(np.NaN)
        r_welsh_J.append(np.NaN)
        r_welsh_K.append(np.NaN)
    k = k+1
    print(k)
end_time = time.time()# total time taken
print(f"Runtime of the program is {end_time - start_time}")


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [7]:
features = pd.DataFrame()
# g filter data
features['g_mean'] = g_mean
features['g_wmean'] = g_wmean
features['g_MAD'] = g_MAD
features['g_IQR'] = g_IQR
features['g_f60'] = g_f60
features['g_f70'] = g_f70
features['g_f80'] = g_f80
features['g_f90'] = g_f90
features['g_skew'] = g_skew
features['g_kurtosis'] = g_kurtosis
features['g_welsh_J'] = g_welsh_J
features['g_welsh_K'] = g_welsh_K

# r filter data
features['r_mean'] = r_mean
features['r_wmean'] = r_wmean
features['r_MAD'] = r_MAD
features['r_IQR'] = r_IQR
features['r_f60'] = r_f60
features['r_f70'] = r_f70
features['r_f80'] = r_f80
features['r_f90'] = r_f90
features['r_skew'] = r_skew
features['r_kurtosis'] = r_kurtosis
features['r_welsh_J'] = r_welsh_J
features['r_welsh_K'] = r_welsh_K

features['label'] = label

features['sourceid'] = SourceID
features.set_index('sourceid')

,g_mean,g_wmean,g_MAD,g_IQR,g_f60,g_f70,g_f80,g_f90,g_skew,g_kurtosis,...,r_IQR,r_f60,r_f70,r_f80,r_f90,r_skew,r_kurtosis,r_welsh_J,r_welsh_K,label
sourceid,,,,,,,,,,,,,,,,,,,,,
487839,18.266840,18.268926,0.030771,0.061090,0.069597,0.087088,0.113279,0.146547,-0.491997,0.573063,...,0.044262,0.052859,0.061141,0.070297,0.079991,-0.033536,-0.674624,0.000644,0.637462,RSCVN
271695,19.925826,19.969903,0.145218,0.286871,0.310142,0.407433,0.478775,0.624174,-0.123830,0.498638,...,0.185196,0.228612,0.295553,0.416920,0.523111,-0.618238,0.225351,0.019193,4.931144,SR
570008,19.161813,19.299435,0.264480,0.503004,0.578186,0.674479,0.751739,0.965016,-0.451999,-0.087775,...,0.333056,0.435942,0.475956,0.501117,0.620676,-0.478345,-0.851582,0.040503,2.874549,RR
437926,16.602879,16.612717,0.091207,0.171326,0.267252,0.362235,0.435667,0.553401,-0.941617,0.653601,...,0.103379,0.127714,0.159584,0.208383,0.406826,-1.236671,1.955880,0.008101,11.155628,SR
331495,19.415213,19.456420,0.166479,0.321408,0.412489,0.473536,0.559076,0.620158,-0.248337,-0.686700,...,0.250072,0.289896,0.340456,0.396845,0.497180,0.080274,-0.926671,0.020252,10.024668,SR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
243198,18.876829,18.889720,0.094601,0.182749,0.219160,0.251572,0.296748,0.428237,0.129996,0.208513,...,0.126784,0.189545,0.223742,0.275309,0.332955,-0.364715,-0.531558,0.008231,5.499722,SR
480687,18.376672,18.423200,0.154530,0.318115,0.357793,0.432657,0.534907,0.634270,0.562683,-0.651363,...,0.301783,0.385128,0.456662,0.528140,0.607849,0.701937,-0.795764,0.036119,4.748564,EW
605788,17.310146,17.321016,0.055948,0.112010,0.142615,0.204332,0.314827,0.421351,1.273680,1.991405,...,0.051256,0.059905,0.083555,0.105848,0.190613,1.619821,2.974895,0.001957,6.090056,SR


In [8]:
features.to_csv('features2.csv',index=False)

In [19]:
df = pd.read_csv('features2.csv', header=None, skiprows=1)
header = pd.read_csv('features2.csv', header=None, nrows=1)
df.columns = header.iloc[0]

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 29 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   g_mean      953 non-null    float64
 1   g_wmean     953 non-null    float64
 2   g_MAD       953 non-null    float64
 3   g_IQR       953 non-null    float64
 4   g_f60       953 non-null    float64
 5   g_f70       953 non-null    float64
 6   g_f80       953 non-null    float64
 7   g_f90       953 non-null    float64
 8   g_skew      953 non-null    float64
 9   g_kurtosis  952 non-null    float64
 10  g_welsh_J   953 non-null    float64
 11  g_welsh_K   953 non-null    float64
 12  r_mean      986 non-null    float64
 13  r_wmean     986 non-null    float64
 14  r_MAD       986 non-null    float64
 15  r_IQR       986 non-null    float64
 16  r_f60       986 non-null    float64
 17  r_f70       986 non-null    float64
 18  r_f80       986 non-null    float64
 19  r_f90       986 non-null    

In [16]:
features['RAdeg'] = labeled_data_sampled.RAdeg.values
features['DEdeg'] = labeled_data_sampled.DEdeg.values
features['ID'] = labeled_data_sampled.ID.values

In [17]:
features.head()

,g_mean,g_wmean,g_MAD,g_IQR,g_f60,g_f70,g_f80,g_f90,g_skew,g_kurtosis,...,r_f90,r_skew,r_kurtosis,r_welsh_J,r_welsh_K,label,sourceid,RAdeg,DEdeg,ID
0,18.266840,18.268926,0.030771,0.061090,0.069597,0.087088,0.113279,0.146547,-0.491997,0.573063,...,0.079991,-0.033536,-0.674624,0.000644,0.637462,RSCVN,487839,295.05785,12.56705,ZTFJ194013.88+123401.3
1,19.925826,19.969903,0.145218,0.286871,0.310142,0.407433,0.478775,0.624174,-0.123830,0.498638,...,0.523111,-0.618238,0.225351,0.019193,4.931144,SR,271695,277.02911,1.31651,ZTFJ182806.98+011859.4
2,19.161813,19.299435,0.264480,0.503004,0.578186,0.674479,0.751739,0.965016,-0.451999,-0.087775,...,0.620676,-0.478345,-0.851582,0.040503,2.874549,RR,570008,301.21524,-9.10789,ZTFJ200451.65-090628.4
3,16.602879,16.612717,0.091207,0.171326,0.267252,0.362235,0.435667,0.553401,-0.941617,0.653601,...,0.406826,-1.236671,1.955880,0.008101,11.155628,SR,437926,291.73444,26.68351,ZTFJ192656.26+264100.6
4,19.415213,19.456420,0.166479,0.321408,0.412489,0.473536,0.559076,0.620158,-0.248337,-0.686700,...,0.497180,0.080274,-0.926671,0.020252,10.024668,SR,331495,284.12455,-5.11396,ZTFJ185629.89-050650.2


In [18]:
features.to_csv('features2.csv',index=False)